In [1]:
import pandas as pd
from snakemake.io import expand
import yaml
import numpy as np
from utils import *
from sm_utils import *
from bc_utils import *

In [2]:
config_file = '../configs/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## 231014 add klue genotype info to our obs table
## use a heuristic to call the genotype of each multiplexed cell


In [235]:
f = '../kallisto/igvf_010/Sublibrary_2/filtered/adata.h5ad'
genotypes = '../klue/igvf_010/Sublibrary_2/genotype_counts.tsv'
ofile = '../klue/igvf_010/Sublibrary_2/adata.h5ad'

In [236]:
def get_genotypes():
    g = ['WSBJ','NZOJ',
         'B6J','NODJ','129S1J',
         'CASTJ','AJ','PWKJ']
    return g
    
def assign_demux_genotype(df):
    """
    Assigns a cell the genotype w/ the maximum of counts
    between the two genotypes that were loaded in the well
    the cell is from.
    
    Parameters:
        df (pandas DataFrame): DF of obs table for each cell w/ 
            klue counts for each genotype and multiplexed genotype
            columns
    """
    genotype_cols = get_genotypes()
    
    # restrict to nuclei w/ genetic multiplexing
    df = df.loc[df.well_type=='Multiplexed'].copy(deep=True)
    
    # fill nans once again
    df[genotype_cols] = df[genotype_cols].fillna(0)
    
    # loop through each multiplexed genotype combo
    # use those genotypes to determine which, 
    # between the two, has the highest counts
    keep_cols = ['mult_genotype',
                 'mult_genotype_1',
                 'mult_genotype_2']+genotype_cols
    df = df[keep_cols]
    temp2 = pd.DataFrame()
    for g in df.mult_genotype.unique().tolist():
        # print(g)
        temp = df.loc[df.mult_genotype==g].copy(deep=True)

        g1 = temp.mult_genotype_1.unique().tolist()
        assert len(g1) == 1
        g1 = g1[0]

        g2 = temp.mult_genotype_2.unique().tolist()
        assert len(g2) == 1
        g2 = g2[0]
    
        # find the best match and report ties if the 
        # values are the same
        temp['new_genotype'] = temp[[g1,g2]].idxmax(axis=1)
        temp.loc[temp[g1]==temp[g2], 'new_genotype'] = 'tie'

        temp2 = pd.concat([temp2, temp], axis=0)

    df = df.merge(temp2['new_genotype'], how='left',
                  left_index=True, right_index=True)
    df = df[['new_genotype']]
    
    assert len(df.loc[df.new_genotype.isnull()].index) == 0

    return df

def merge_kallisto_klue(f, genotypes, ofile):
    """
    Merge in the klue results with the kallisto results. Use
    a heuristic (which should be changeable / is subject to change)
    to determine the genotype assignment to each cell
    """
    
    adata = sc.read(f)
    df = pd.read_csv(genotypes, sep='\t')
    df.set_index('cellID', inplace=True)
    
    # make sure we won't dupe any cols
    assert len(set(df.columns.tolist())&set(adata.obs.columns.tolist())) == 0    
    
    # merge in first; this way we have access to the genotypes that should
    # be in each well
    adata.obs = adata.obs.merge(df,
                                how='left', 
                                left_index=True,
                                right_index=True)
    
    # assign genotype for multiplexed wells
    df = adata.obs.copy(deep=True) 
    df = assign_demux_genotype(df)   
    
    # merge in w/ adata and replace old values in "Genotype" 
    # column for multiplexed wells with the klue results
    adata.obs = adata.obs.merge(df, 
                                how='left',
                                left_index=True,
                                right_index=True)
    inds = adata.obs.loc[adata.obs.well_type=='Multiplexed'].index
    adata.obs.Genotype = adata.obs.Genotype.astype('str')
    adata.obs.loc[inds, 'Genotype'] = adata.obs.loc[inds, 'new_genotype']
    adata.obs.drop('new_genotype', axis=1, inplace=True)
    
    adata.write(ofile)


In [237]:
merge_kallisto_klue(f, genotypes, ofile)

In [229]:
# adata = sc.read(f)
# df = pd.read_csv(genotypes, sep='\t')
# df.set_index('cellID', inplace=True)

In [230]:
# assert len(set(df.columns.tolist())&set(adata.obs.columns.tolist())) == 0

In [231]:
# # merge in first; this way we have access to the genotypes that should
# # be in each well
# adata.obs = adata.obs.merge(df,
#                             how='left', 
#                             left_index=True,
#                             right_index=True)

In [220]:
# def get_genotypes():
#     g = ['WSBJ','NZOJ',
#          'B6J','NODJ','129S1J',
#          'CASTJ','AJ','PWKJ']
#     return g

In [232]:
# df = adata.obs.copy(deep=True)

In [233]:
# def assign_demux_genotype(df):
#     """
#     Parameters:
#         df (pandas DataFrame): DF of obs table for each cell w/ 
#             klue counts for each genotype and multiplexed genotype
#             columns
#     """
#     genotype_cols = get_genotypes()
    
#     # restrict to nuclei w/ genetic multiplexing
#     df = df.loc[df.well_type=='Multiplexed'].copy(deep=True)
    
#     # fill nans once again
#     df[genotype_cols] = df[genotype_cols].fillna(0)
    
#     # loop through each multiplexed genotype combo
#     # use those genotypes to determine which, 
#     # between the two, has the highest counts
#     keep_cols = ['mult_genotype',
#                  'mult_genotype_1',
#                  'mult_genotype_2']+genotype_cols
#     df = df[keep_cols]
#     temp2 = pd.DataFrame()
#     for g in df.mult_genotype.unique().tolist():
#         # print(g)
#         temp = df.loc[df.mult_genotype==g].copy(deep=True)

#         g1 = temp.mult_genotype_1.unique().tolist()
#         assert len(g1) == 1
#         g1 = g1[0]

#         g2 = temp.mult_genotype_2.unique().tolist()
#         assert len(g2) == 1
#         g2 = g2[0]
    
#         # find the best match and report ties if the 
#         # values are the same
#         temp['new_genotype'] = temp[[g1,g2]].idxmax(axis=1)
#         temp.loc[temp[g1]==temp[g2], 'new_genotype'] = 'tie'

#         temp2 = pd.concat([temp2, temp], axis=0)

#     df = df.merge(temp2['new_genotype'], how='left',
#                   left_index=True, right_index=True)
#     df = df[['new_genotype']]
    
#     assert len(df.loc[df.new_genotype.isnull()].index) == 0

#     return df

In [234]:
# df = assign_demux_genotype(df)

In [227]:
# merge in w/ adata and replace old values in "Genotype" 
# column for multiplexed wells with the klue results
adata.obs = adata.obs.merge(df, 
                            how='left',
                            left_index=True,
                            right_index=True)
inds = adata.obs.loc[adata.obs.well_type=='Multiplexed'].index
adata.obs.Genotype = adata.obs.Genotype.astype('str')
adata.obs.loc[inds, 'Genotype'] = adata.obs.loc[inds, 'new_genotype']
adata.obs.drop('new_genotype', axis=1, inplace=True)


KeyError: 'new_genotype'

In [226]:
adata.obs.loc[adata.obs.Genotype.isnull()]

,bc,bc1_sequence,bc2_sequence,bc3_sequence,subpool,n_counts,bc1_well,bc2_well,bc3_well,Mouse_Tissue_ID,...,mult_genotype,WSBJ,NZOJ,B6J,NODJ,129S1J,CASTJ,AJ,PWKJ,new_genotype
cellID,,,,,,,,,,,,,,,,,,,,,


In [203]:
adata.obs.dtypes

bc                       object
bc1_sequence           category
bc2_sequence           category
bc3_sequence           category
subpool                category
n_counts                float64
bc1_well               category
bc2_well               category
bc3_well               category
Mouse_Tissue_ID        category
alias_tissue1          category
alias_tissue2          category
alias_tissue3          category
alias_tissue4          category
plate                  category
Protocol               category
Chemistry              category
well_type              category
Row                    category
Column                    int64
Genotype               category
Notes                  category
Multiplexed_sample1    category
Multiplexed_sample2    category
Tissue                 category
Tissue_ID                 int64
Tissue1_ontology_id    category
Tissue2_ontology_id    category
Sex                    category
Replicate                 int64
DOB                    category
Age_week

In [140]:
# genotype_cols = get_genotypes()
# keep_cols = ['mult_genotype',
#              'mult_genotype_1',
#              'mult_genotype_2']+genotype_cols
# df = df[keep_cols]
# temp2 = pd.DataFrame()
# for g in df.mult_genotype.unique().tolist():
#     print(g)
#     temp = df.loc[df.mult_genotype==g]
    
#     g1 = temp.mult_genotype_1.unique().tolist()
#     assert len(g1) == 1
#     g1 = g1[0]
    
#     g2 = temp.mult_genotype_2.unique().tolist()
#     assert len(g2) == 1
#     g2 = g2[0]
    
#     temp['new_genotype'] = temp[[g1,g2]].idxmax(axis=1)
#     temp.loc[temp[g1]==temp[g2], 'new_genotype'] = 'tie'
    
#     temp2 = pd.concat([temp2, temp], axis=0)
    
# df = df.merge(temp2['new_genotype'], how='left',
#               left_index=True, right_index=True)
# # # temp
# # print(g)
# # print(g1)

# # print(g2)

WSBJ_NZOJ
AJ_PWKJ
129S1J_CASTJ
B6J_NODJ


/Users/fairliereese/miniconda3/envs/base_clone/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fairliereese/miniconda3/envs/base_clone/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [141]:
df.head()

,mult_genotype,mult_genotype_1,mult_genotype_2,WSBJ,NZOJ,B6J,NODJ,129S1J,CASTJ,AJ,PWKJ,new_genotype
cellID,,,,,,,,,,,,
G10_A2_A2_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
H11_A2_A2_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D11_A2_A2_Sublibrary_2_igvf_010,AJ_PWKJ,AJ,PWKJ,0.0,0.0,0.0,0.0,0.0,0.0,20.0,529.0,PWKJ
C9_B3_A2_Sublibrary_2_igvf_010,AJ_PWKJ,AJ,PWKJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C12_B3_A2_Sublibrary_2_igvf_010,AJ_PWKJ,AJ,PWKJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
df.loc[(df.new_genotype.notnull())&(df.new_genotype!='tie')]

,mult_genotype,mult_genotype_1,mult_genotype_2,WSBJ,NZOJ,B6J,NODJ,129S1J,CASTJ,AJ,PWKJ,new_genotype
cellID,,,,,,,,,,,,
D11_A2_A2_Sublibrary_2_igvf_010,AJ_PWKJ,AJ,PWKJ,0.0,0.0,0.0,0.0,0.0,0.0,20.0,529.0,PWKJ
F12_F10_A2_Sublibrary_2_igvf_010,129S1J_CASTJ,129S1J,CASTJ,0.0,0.0,0.0,0.0,16.0,602.0,283.0,295.0,CASTJ
F11_F12_A2_Sublibrary_2_igvf_010,129S1J_CASTJ,129S1J,CASTJ,0.0,0.0,0.0,0.0,158.0,388.0,300.0,202.0,CASTJ
C12_C9_A2_Sublibrary_2_igvf_010,AJ_PWKJ,AJ,PWKJ,0.0,0.0,0.0,0.0,500.0,602.0,30.0,1130.0,PWKJ
F12_D1_A2_Sublibrary_2_igvf_010,129S1J_CASTJ,129S1J,CASTJ,0.0,0.0,0.0,0.0,19.0,538.0,0.0,0.0,CASTJ
...,...,...,...,...,...,...,...,...,...,...,...,...
D11_E10_F8_Sublibrary_2_igvf_010,AJ_PWKJ,AJ,PWKJ,0.0,0.0,0.0,0.0,568.0,97.0,766.0,15.0,AJ
F9_F1_F8_Sublibrary_2_igvf_010,129S1J_CASTJ,129S1J,CASTJ,0.0,0.0,0.0,0.0,555.0,9.0,517.0,61.0,129S1J
F10_F3_F8_Sublibrary_2_igvf_010,129S1J_CASTJ,129S1J,CASTJ,0.0,0.0,0.0,0.0,19.0,881.0,372.0,407.0,CASTJ


In [143]:
len(df.loc[df.new_genotype=='tie'])

2995

In [ ]:
# len(df.loc[(df.new_genotype=='tie')&(df.])

In [132]:
temp.loc[temp.WSBJ==temp.NZOJ]

,mult_genotype,mult_genotype_1,mult_genotype_2,WSBJ,NZOJ,B6J,NODJ,129S1J,CASTJ,AJ,PWKJ,new_genotype
cellID,,,,,,,,,,,,
H10_G4_A2_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,494.0,77.0,521.0,109.0,tie
H10_G9_A2_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,519.0,99.0,589.0,88.0,tie
G9_A7_A2_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,0.0,0.0,461.0,57.0,tie
G10_C11_A2_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,0.0,0.0,493.0,62.0,tie
H9_A11_A2_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,0.0,0.0,475.0,93.0,tie
...,...,...,...,...,...,...,...,...,...,...,...,...
G10_D12_F8_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,0.0,0.0,511.0,54.0,tie
H9_D12_F8_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,0.0,0.0,504.0,83.0,tie
H11_E4_F8_Sublibrary_2_igvf_010,WSBJ_NZOJ,WSBJ,NZOJ,0.0,0.0,0.0,0.0,0.0,0.0,589.0,47.0,tie


In [134]:
temp.loc[temp.new_genotype == 'WSB']

,mult_genotype,mult_genotype_1,mult_genotype_2,WSBJ,NZOJ,B6J,NODJ,129S1J,CASTJ,AJ,PWKJ,new_genotype
cellID,,,,,,,,,,,,


In [97]:
adata.obs.Genotype.unique()

['WSBJ/NZOJ', 'AJ', 'AJ/PWKJ', 'NODJ', 'B6J', ..., 'NZOJ', '129S1J/CASTJ', 'B6J/NODJ', 'WSBJ', 'CASTJ']
Length: 12
Categories (12, object): ['129S1J', '129S1J/CASTJ', 'AJ', 'AJ/PWKJ', ..., 'NZOJ', 'PWKJ', 'WSBJ', 'WSBJ/NZOJ']

## 231014 filter / do some other reformatting for klue output

In [69]:
import scanpy as sc

In [70]:
files = ['/Users/fairliereese/mortazavi_lab/bin/igvf_pipeline/klue/igvf_010/Sublibrary_2/129S1J_CASTJ/filtered/adata.h5ad', 
         '/Users/fairliereese/mortazavi_lab/bin/igvf_pipeline/klue/igvf_010/Sublibrary_2/AJ_PWKJ/filtered/adata.h5ad']

In [ ]:
def get_genotype_counts(files, ofile):
    """
    Given a list of klue anndata objects, merge 
    the genotype counts together for each cell and output
    as a tsv
    """
    for i, f in enumerate(files):
        adata = sc.read(f)
        if i == 0:
            df = adata.to_df()
        else:
            df = df.merge(adata.to_df(),
                          how='outer',
                          left_index=True,
                          right_index=True)
    df.to_csv(ofile, index=False, sep='\t')

In [77]:
for i, f in enumerate(files):
    adata = sc.read(f)
    if i == 0:
        df = adata.to_df()
    else:
        df = df.merge(adata.to_df(),
                      how='outer',
                      left_index=True,
                      right_index=True)

In [78]:
df.head()

genotype,129S1J,CASTJ,AJ,PWKJ
cellID,,,,
A10_A12_F9_Sublibrary_2_igvf_010,512.0,86.0,643.0,87.0
A10_A1_D9_Sublibrary_2_igvf_010,NaN,NaN,430.0,80.0
A10_A1_E7_Sublibrary_2_igvf_010,716.0,115.0,1077.0,182.0
A10_A4_C3_Sublibrary_2_igvf_010,477.0,90.0,564.0,75.0
A10_A4_G3_Sublibrary_2_igvf_010,438.0,90.0,583.0,113.0


In [80]:
# meta.head()

## 231014 How to ask for all klue outputs?

In [3]:
config_tsv = '../configs/test_4.tsv'
sample_csv = '../configs/sample_metadata.csv'
kit = 'WT_mega'
chemistry = 'v2'

In [4]:
# read in config / analysis spec
df = parse_config(config_tsv)
bc_df = get_bc1_matches(kit, chemistry)
sample_df = parse_sample_df(sample_csv)

mult_genotype_1s = sample_df.loc[sample_df.mult_genotype_1.notnull(), 'mult_genotype_1'].unique().tolist()
mult_genotype_2s = sample_df.loc[sample_df.mult_genotype_2.notnull(), 'mult_genotype_2'].unique().tolist()

In [15]:
temp = df.merge(sample_df, on='plate', how='inner')
temp = temp.loc[temp.mult_genotype.notnull()]


In [16]:
temp.head()

,fastq,subpool,plate,lane,run,path,path2,r2_fastq,Mouse_Tissue_ID,alias_tissue1,...,Age_weeks,Age_days,Body_weight_g,Estrus_cycle,Dissection_date,Dissection_time,Tissue_weight_mg,mult_genotype_1,mult_genotype_2,mult_genotype
64,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,Sublibrary_2,igvf_010,L001,1,/dfs7/samlab/seyedam/IGVF/igvf_010/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,AB_M1_16,ali-mortazavi:017_B6J_10M_16_UBERON_0001388,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
65,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,Sublibrary_2,igvf_010,L001,1,/dfs7/samlab/seyedam/IGVF/igvf_010/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,AB_F2_16,ali-mortazavi:018_B6J_10F_16_UBERON_0001388,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
66,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,Sublibrary_2,igvf_010,L001,1,/dfs7/samlab/seyedam/IGVF/igvf_010/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,AB_M2_16,ali-mortazavi:019_B6J_10M_16_UBERON_0001388,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
67,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,Sublibrary_2,igvf_010,L001,1,/dfs7/samlab/seyedam/IGVF/igvf_010/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,AB_F1_16,ali-mortazavi:016_B6J_10F_16_UBERON_0001388,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
68,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,Sublibrary_2,igvf_010,L001,1,/dfs7/samlab/seyedam/IGVF/igvf_010/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/,/dfs7/samlab/seyedam/IGVF/igvf_010/nova1/Subli...,AB_M3_16,ali-mortazavi:021_B6J_10M_16_UBERON_0001388,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ


In [19]:
# # def get_subset_mult_genotypes(df, sample_df):
# temp = df.merge(sample_df, on='plate', how='inner')
# temp = temp.loc[temp.mult_genotype.notnull()]
# mult_genotype_1s = temp.mult_genotype_1.tolist()
# mult_genotype_2s = temp.mult_genotype_2.tolist()
# # return mult_genotype_1s, mult_genotype_2s

def get_subset_mult_genotypes(df, sample_df, col):
    temp = df.merge(sample_df, on='plate', how='inner')
    temp = temp.loc[temp.mult_genotype.notnull()]
    entries = temp[col].tolist()
    return entries

In [20]:
entries = get_subset_mult_genotypes(df, sample_df, 'mult_genotype_1')

In [12]:
entries

['B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1J',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'WSBJ',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'B6J',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 'AJ',
 '129S1J',
 '129S1J',
 '129S1J',
 '129S1

## 231014 adding multiplexed genotype as fields to the sample metadata

In [17]:
sample_csv = '../configs/sample_metadata.csv'


In [49]:
def parse_sample_df(fname):
    df = pd.read_csv(fname)

    # add multiplexed genotypes if relevant
    g_cols = ['Multiplexed_Genotype_1', 'Multiplexed_Genotype_2']
    df[g_cols] = df.Genotype.str.split('/', expand=True)

    # adjust single-genotype wells
    df.loc[df.well_type=='Single', g_cols] = np.nan

    # add a multiplexed genotype column
    inds = df.loc[df.well_type=='Multiplexed'].index
    df['mult_genotype'] = np.nan
    df.loc[inds, 'mult_genotype'] = df.loc[inds, g_cols[0]]+'_'+\
                                   df.loc[inds, g_cols[1]]

    # checks
    for g in g_cols:
        assert len(df.loc[(df.well_type=='Single')&(df[g].notnull())].index) == 0

    return df

In [50]:
df = parse_sample_df(sample_csv)

In [51]:
df.loc[df.well_type=='Multiplexed']

,Mouse_Tissue_ID,alias_tissue1,alias_tissue2,alias_tissue3,alias_tissue4,plate,Protocol,Chemistry,bc1_well,well_type,...,Age_weeks,Age_days,Body_weight_g,Estrus_cycle,Dissection_date,Dissection_time,Tissue_weight_mg,Multiplexed_Genotype_1,Multiplexed_Genotype_2,mult_genotype
64,AB_M1_06,ali-mortazavi:017_B6J_10M_06_UBERON_0000948,ali-mortazavi:067_NODJ_10M_06_UBERON_0000948,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A10,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
65,AB_F2_06,ali-mortazavi:018_B6J_10F_06_UBERON_0000948,ali-mortazavi:068_NODJ_10F_06_UBERON_0000948,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A11,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
66,AB_M2_06,ali-mortazavi:019_B6J_10M_06_UBERON_0000948,ali-mortazavi:069_NODJ_10M_06_UBERON_0000948,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A12,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
67,AB_F1_06,ali-mortazavi:016_B6J_10F_06_UBERON_0000948,ali-mortazavi:066_NODJ_10F_06_UBERON_0000948,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A9,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
68,AB_M3_06,ali-mortazavi:021_B6J_10M_06_UBERON_0000948,ali-mortazavi:071_NODJ_10M_06_UBERON_0000948,NaN,NaN,igvf_003,Parse_WT_Mega,v2,B10,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,B6J,NODJ,B6J_NODJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,GH_F1_03,ali-mortazavi:058_WSBJ_10F_03_NTR_0000646,ali-mortazavi:092_CASTJ_10F_03_NTR_0000646,ali-mortazavi:058_WSBJ_10F_03_NTR_0000750,ali-mortazavi:092_CASTJ_10F_03_NTR_0000750,igvf_011,Parse_WT_Mega,v2,G9,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,WSBJ,CASTJ,WSBJ_CASTJ
860,GH_M3_03,ali-mortazavi:061_WSBJ_10M_03_NTR_0000646,ali-mortazavi:051_NZOJ_10M_03_NTR_0000646,ali-mortazavi:061_WSBJ_10M_03_NTR_0000750,ali-mortazavi:051_NZOJ_10M_03_NTR_0000750,igvf_011,Parse_WT_Mega,v2,H10,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,WSBJ,NZOJ,WSBJ_NZOJ
861,GH_F4_03,ali-mortazavi:096_WSBJ_10F_03_NTR_0000646,ali-mortazavi:052_NZOJ_10F_03_NTR_0000646,ali-mortazavi:096_WSBJ_10F_03_NTR_0000750,ali-mortazavi:052_NZOJ_10F_03_NTR_0000750,igvf_011,Parse_WT_Mega,v2,H11,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,WSBJ,NZOJ,WSBJ_NZOJ
862,GH_M4_03,ali-mortazavi:063_WSBJ_10M_03_NTR_0000646,ali-mortazavi:053_NZOJ_10M_03_NTR_0000646,ali-mortazavi:063_WSBJ_10M_03_NTR_0000750,ali-mortazavi:053_NZOJ_10M_03_NTR_0000750,igvf_011,Parse_WT_Mega,v2,H12,Multiplexed,...,10,NaN,NaN,NaN,NaN,NaN,NaN,WSBJ,NZOJ,WSBJ_NZOJ


In [40]:
# df

In [28]:
g ='Multiplexed_Genotype_1'
df.loc[(df.well_type=='Single')&(df[g].notnull())]

,Mouse_Tissue_ID,alias_tissue1,alias_tissue2,alias_tissue3,alias_tissue4,plate,Protocol,Chemistry,bc1_well,well_type,...,DOB,Age_weeks,Age_days,Body_weight_g,Estrus_cycle,Dissection_date,Dissection_time,Tissue_weight_mg,Multiplexed_Genotype_1,Multiplexed_Genotype_2


In [42]:
df.loc[df.well_type=='Single']

,Mouse_Tissue_ID,alias_tissue1,alias_tissue2,alias_tissue3,alias_tissue4,plate,Protocol,Chemistry,bc1_well,well_type,...,Age_weeks,Age_days,Body_weight_g,Estrus_cycle,Dissection_date,Dissection_time,Tissue_weight_mg,Multiplexed_Genotype_1,Multiplexed_Genotype_2,mult_genotype
0,016_B6J_10F_03,ali-mortazavi:016_B6J_10F_03_NTR_0000646,ali-mortazavi:016_B6J_10F_03_NTR_0000750,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A1,Single,...,10,72.0,21.1,Diestrus,10/27/22,9:01 AM,141.0,NaN,NaN,NaN
1,017_B6J_10M_03,ali-mortazavi:017_B6J_10M_03_NTR_0000646,ali-mortazavi:017_B6J_10M_03_NTR_0000750,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A2,Single,...,10,72.0,26.3,NaN,10/27/22,11:44 AM,141.0,NaN,NaN,NaN
2,018_B6J_10F_03,ali-mortazavi:018_B6J_10F_03_NTR_0000646,ali-mortazavi:018_B6J_10F_03_NTR_0000750,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A3,Single,...,10,72.0,21.0,Proestrus,10/27/22,9:38 AM,143.0,NaN,NaN,NaN
3,019_B6J_10M_03,ali-mortazavi:019_B6J_10M_03_NTR_0000646,ali-mortazavi:019_B6J_10M_03_NTR_0000750,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A4,Single,...,10,72.0,29.0,NaN,10/27/22,12:12 PM,141.0,NaN,NaN,NaN
4,020_B6J_10F_03,ali-mortazavi:020_B6J_10F_03_NTR_0000646,ali-mortazavi:020_B6J_10F_03_NTR_0000750,NaN,NaN,igvf_003,Parse_WT_Mega,v2,A5,Single,...,10,72.0,18.5,Diestrus,10/27/22,10:14 AM,149.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,163_B6NODF1J_10M_20,ali-mortazavi:163_B6NODF1J_10M_20_CL_2000001,NaN,NaN,NaN,igvf_013,Parse_WT_Mega,v2,D12,Single,...,10,NaN,25.9,NaN,7/24/23,1:38 PM,NaN,NaN,NaN,NaN
1052,166_B6NODF1J_10F_20,ali-mortazavi:166_B6NODF1J_10F_20_CL_2000001,NaN,NaN,NaN,igvf_013,Parse_WT_Mega,v2,E12,Single,...,10,NaN,20.5,NaN,7/24/23,11:10 AM,NaN,NaN,NaN,NaN
1053,165_B6NODF1J_10M_20,ali-mortazavi:165_B6NODF1J_10M_20_CL_2000001,NaN,NaN,NaN,igvf_013,Parse_WT_Mega,v2,F12,Single,...,10,NaN,26.0,NaN,7/24/23,1:58 PM,NaN,NaN,NaN,NaN
1054,168_B6NODF1J_10F_20,ali-mortazavi:168_B6NODF1J_10F_20_CL_2000001,NaN,NaN,NaN,igvf_013,Parse_WT_Mega,v2,G12,Single,...,10,NaN,18.9,NaN,7/24/23,11:41 AM,NaN,NaN,NaN,NaN


## 231006 tissue level agg

In [7]:
config_tsv = '../configs/test_2.tsv'
sample_csv = '../configs/sample_metadata.csv'

In [17]:
df = parse_config(config_tsv)
sample_df = pd.read_csv(sample_csv)
wc = {'tissue': 'PBMC'}

In [30]:
def get_tissue_adatas(df, sample_df, wc, cfg_entry):
    
    # limit to input tissue
    temp_sample = sample_df.copy(deep=True)
    print(len(temp_sample.index))
    temp_sample = temp_sample.loc[temp_sample.Tissue==wc['tissue']]
    print(len(temp_sample.index))
    
    # merge this stuff in with the fastq df
    fastq_df = df.copy(deep=True)
    temp = fastq_df.merge(sample_df, on='plate', how='inner')
    
    # get the plate / subpool / sample info for this tissue
    plates = temp.plate().tolist()
    subpools = temp.subpool.tolist()
    samples = temp.tolist()


In [31]:
get_tissue_adatas(df, sample_df, wc, 'test')

1056
96
['igvf_013']
1
['Sublibrary_2', 'Sublibrary_3']
2
['016_B6J_10F_20', '017_B6J_10M_20', '018_B6J_10F_20', '019_B6J_10M_20', '020_B6J_10F_20', '021_B6J_10M_20', '024_B6J_10F_20', '025_B6J_10M_20', '066_NODJ_10F_20', '067_NODJ_10M_20', '070_NODJ_10F_20', '069_NODJ_10M_20', '072_NODJ_10F_20', '073_NODJ_10M_20', '074_NODJ_10F_20', '075_NODJ_10M_20', '026_AJ_10F_20', '029_AJ_10M_20', '028_AJ_10F_20', '031_AJ_10M_20', '032_AJ_10F_20', '033_AJ_10M_20', '034_AJ_10F_20', '035_AJ_10M_20', '076_PWKJ_10F_20', '077_PWKJ_10M_20', '078_PWKJ_10F_20', '079_PWKJ_10M_20', '080_PWKJ_10F_20', '083_PWKJ_10M_20', '082_PWKJ_10F_20', '085_PWKJ_10M_20', '036_129S1J_10F_20', '037_129S1J_10M_20', '038_129S1J_10F_20', '041_129S1J_10M_20', '040_129S1J_10F_20', '043_129S1J_10M_20', '044_129S1J_10F_20', '045_129S1J_10M_20', '086_CASTJ_10F_20', '087_CASTJ_10M_20', '090_CASTJ_10F_20', '091_CASTJ_10M_20', '092_CASTJ_10F_20', '093_CASTJ_10M_20', '094_CASTJ_10F_20', '095_CASTJ_10M_20', '058_WSBJ_10F_20', '057_WSBJ_

## 231006 more bc stuff

In [7]:
kit = 'WT_mega'
chemistry = 'v2'
bc = 3

In [8]:
df = get_bcs(bc, kit, chemistry)

In [9]:
df

,bc3,well
0,AACGTGAT,A1
1,AAACATCG,A2
2,ATGCCTAA,A3
3,AGTGGTCA,A4
4,ACCACTGT,A5
...,...,...
91,GAACAGGC,H8
92,GACAGTGC,H9
93,GAGTTAGC,H10
94,GATGAATC,H11


In [3]:
kit = 'WT_mega'
chemistry = 'v2'

In [6]:
bc_df = get_bc1_matches(kit, chemistry)

In [7]:
bc_df

,bc1_dt,well,bc1_randhex
0,CATTCCTA,A1,CATCATCC
1,CTTCATCA,A2,CTGCTTTG
2,CCTATATC,A3,CTAAGGGA
3,ACATTTAC,A4,GCTTATAG
4,ACTTAGCT,A5,TCTGATCC
...,...,...,...
91,ATTAGGCT,H8,GTGTGTGT
92,GCCTTTCA,H9,TATGCTTC
93,ATTCTAGG,H10,ATGGTGTT
94,CCTTACAT,H11,GAATAATG


In [68]:
fname = '../configs/test_2.tsv'

def parse_config(fname):
    df = pd.read_csv(fname, sep='\t')
    df['path'] = df.fastq.str.rsplit('/', n=2, expand=True)[0]+'/'
    df['path2'] = df.fastq.str.rsplit('/', n=1, expand=True)[0]+'/'
    df['r2_fastq'] = df.fastq.str.replace('_R1_', '_R2_')
    return df

In [76]:
def get_subpool_fastqs(wc, df, how, read=None):
    """
    Get list of fastqs from the same subpool. Can
    either return as a Python list of strings or a
    formatted string list read to pass to a shell cmd.

    Parameters:
        how (str): {'str', 'list'}. 'list' will return
            Python list of str var. 'str' will return
            Python string
    """
    temp = df.copy(deep=True)
    temp = temp.loc[(temp.plate==wc['plate'])&\
                    (temp.subpool==wc['subpool'])]

    if how == 'list':
        reads = [read for i in range(len(temp.index))]
        return expand(expand(config['raw']['fastq'],
                        zip,
                        sample=temp['sample'].tolist(),
                        lane=temp['lane'].tolist(),
                        allow_missing=True),
                        read=read,
                        plate=wc['plate'],
                        subpool=wc['subpool'])

    elif how == 'str':
        r1s = expand(expand(config['raw']['fastq'],
                        zip,
                        sample=temp['sample'].tolist(),
                        lane=temp['lane'].tolist(),
                        allow_missing=True),
                        read='R1',
                        plate=wc['plate'],
                        subpool=wc['subpool'])
        r2s = expand(expand(config['raw']['fastq'],
                        zip,
                        sample=temp['sample'].tolist(),
                        lane=temp['lane'].tolist(),
                        allow_missing=True),
                        read='R2',
                        plate=wc['plate'],
                        subpool=wc['subpool'])
        fastq_str = ''
        for r1, r2 in zip(r1s, r2s):
            fastq_str+=f' {r1} {r2}'
        return fastq_str

In [86]:
def get_df_info(wc, df, col):
    temp = df.copy(deep=True)
    temp = temp.loc[(temp.plate==wc['plate'])&\
                    (temp.subpool==wc['subpool'])&\
                    (temp['sample']==wc['sample'])&\
                    (temp.lane==wc['lane'])]
    assert len(temp.index) == 1
    return temp[col].values[0]

In [87]:
df = parse_config(fname)
wc = {'plate': 'igvf_013', 'subpool': 'Sublibrary_2', 'sample': 'S1', 'lane': 'L001', 'read': 'R1'}

In [88]:
get_df_info(wc, df, 'r2_fastq')

'/dfs7/samlab/seyedam/IGVF/igvf_013/nova1/Sublibrary_2_S1_L001_R2_001.fastq.gz'

In [78]:
get_subpool_fastqs(wc, df, 'list', read='R2')

['S1', 'S1', 'S1', 'S1']
['L001', 'L002', 'L003', 'L004']
R2
igvf_013
Sublibrary_2


['fastq/igvf_013_Sublibrary_2_S1_L001_R2_001.fastq.gz',
 'fastq/igvf_013_Sublibrary_2_S1_L002_R2_001.fastq.gz',
 'fastq/igvf_013_Sublibrary_2_S1_L003_R2_001.fastq.gz',
 'fastq/igvf_013_Sublibrary_2_S1_L004_R2_001.fastq.gz']

In [79]:
get_subpool_fastqs(wc, df, 'str')

['S1', 'S1', 'S1', 'S1']
['L001', 'L002', 'L003', 'L004']
None
igvf_013
Sublibrary_2


' fastq/igvf_013_Sublibrary_2_S1_L001_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L001_R2_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L002_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L002_R2_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L003_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L003_R2_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L004_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L004_R2_001.fastq.gz'

In [23]:
df.r2_fastq.tolist()[:2]

['/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L001_R2_001.fastq.gz',
 '/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L002_R2_001.fastq.gz']